In this notebook, I am collecting everything one needs to run densepose within the google colab environment.

First, we'll mount our google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Next, we'll install Anaconda2 to then use it to install caffe2, install the COCO API and finally densepose. For the latter, we have to make sure we are using the correct & latest "env.py" from the detectron utils. Finally, we'll fetch the densepose data.

In [0]:
import os

!wget -c https://repo.continuum.io/archive/Anaconda2-5.3.0-Linux-x86_64.sh
!chmod +x Anaconda2-5.3.0-Linux-x86_64.sh
!bash ./Anaconda2-5.3.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python2.7/site-packages/')

!conda install -y pytorch-nightly -c pytorch
!pip install argparse
!pip install pyhamcrest
!pip install protobuf
!pip install future
!pip install pydot
!pip install cython scikit-image easydict

!python2 -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'
!python2 -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

COCOAPI='/usr/local/cocoapi/'
!git clone https://github.com/cocodataset/cocoapi.git $COCOAPI
  
os.chdir('/usr/local/cocoapi/PythonAPI/')
!python2 setup.py install --user

DENSEPOSE='/usr/local/densepose/'
!git clone https://github.com/facebookresearch/densepose $DENSEPOSE
os.chdir('/usr/local/densepose/detectron/utils/')
!rm env.py
!wget https://raw.githubusercontent.com/facebookresearch/Detectron/master/detectron/utils/env.py
!pip install -r $DENSEPOSE/requirements.txt
os.chdir('/usr/local/densepose/')
!make
!python $DENSEPOSE/detectron/tests/test_spatial_narrow_as_op.py

os.chdir('/usr/local/densepose/DensePoseData/')
!bash get_densepose_uv.sh
!bash get_eval_data.sh

Now we're ready to infer the positions of bodies in images:

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --image-ext jpg \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    /content/gdrive/My\ Drive/skopdaube/benjaminDanceTest/

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --image-ext jpg \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    /content/gdrive/My\ Drive/skopdaube/benjDance720/

Copy the visualisations of the detected bodies into our drive

In [0]:
cp DensePoseData/infer_out/*IUV.png /content/gdrive/My\ Drive/skopdaube/testOut/

Here I am trying to use trrahul's code that is supposed to work directly on video files instead of the single frames. So far, I failed to install ffmpeg though.

In [0]:

cd /usr/local/densepose/tools/
!wget https://raw.githubusercontent.com/trrahul/densepose-video/master/infer_vid.py
!wget https://raw.githubusercontent.com/trrahul/densepose-video/master/vis.py

In [0]:
!python2 tools/infer_vid.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    --input-file /content/gdrive/My\ Drive/skopdaube/videoTest/test1.mov


In [9]:
cd DensePoseData/infer_out/

/usr/local/densepose/DensePoseData/infer_out


In [10]:
ls

'Benjamin Skop000_INDS.png'  'Benjamin Skop120.jpg.pdf'
'Benjamin Skop000_IUV.png'   'Benjamin Skop121_INDS.png'
'Benjamin Skop000.jpg.pdf'   'Benjamin Skop121_IUV.png'
'Benjamin Skop001_INDS.png'  'Benjamin Skop121.jpg.pdf'
'Benjamin Skop001_IUV.png'   'Benjamin Skop122_INDS.png'
'Benjamin Skop001.jpg.pdf'   'Benjamin Skop122_IUV.png'
'Benjamin Skop002_INDS.png'  'Benjamin Skop122.jpg.pdf'
'Benjamin Skop002_IUV.png'   'Benjamin Skop123_INDS.png'
'Benjamin Skop002.jpg.pdf'   'Benjamin Skop123_IUV.png'
'Benjamin Skop003_INDS.png'  'Benjamin Skop123.jpg.pdf'
'Benjamin Skop003_IUV.png'   'Benjamin Skop124_INDS.png'
'Benjamin Skop003.jpg.pdf'   'Benjamin Skop124_IUV.png'
'Benjamin Skop004_INDS.png'  'Benjamin Skop124.jpg.pdf'
'Benjamin Skop004_IUV.png'   'Benjamin Skop125_INDS.png'
'Benjamin Skop004.jpg.pdf'   'Benjamin Skop125_IUV.png'
'Benjamin Skop005_INDS.png'  'Benjamin Skop125.jpg.pdf'
'Benjamin Skop005_IUV.png'   'Benjamin Skop126_INDS.png'
'Benjamin Skop005.jpg.pdf'   'Benjamin Sko